In [1]:
#import modules
import xarray as xr
import numpy as np
import pandas as pd
import plotly.graph_objects as go

#import local module for analysis
from WRFChemToolkit.analysis import functions as anly
from WRFChemToolkit.plots import functions as plot

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings
warnings.filterwarnings('ignore')

In [2]:
datapath ='/geos/d21/s1878599/BB_season2017_16cores_holmes/wrfout*'
#Merge datasets
ds = xr.open_mfdataset(datapath)

In [3]:
def pm25_component_of(dataset, species):
    """
    Sum up the species values for bin sizes (from 1 to 3) making up PM25.
    Add the total contribution of the species to PM25 concentration.
    TODO change units from mixing ratio to ug/m3.

    :param species: name of the chemical species
    :type species: string.
    :return:
    :rtype: xarray DataArray.
    """
    return dataset[species + '_a01'] + dataset[species + '_a02'] + dataset[
        species + '_a03']

In [4]:
def var_space_mean(ds):
 """
  Make the average over '' dimension of a datarray.

  :param da:
    datarray to be averaged.
  :type da: xarray DataArray.
  :return:
    Time averaged da.
  :rtype: xarray DataSet.
 """
 return xr.Dataset(dict(ds.mean(dim= ['south_north','west_east'],
                                   keep_attrs=True).data_vars), coords=dict(ds.coords))

In [5]:
#sum up components
ds['so4_pm25'] = pm25_component_of(ds,'so4')
ds['nh4_pm25'] = pm25_component_of(ds,'nh4')
ds['no3_pm25'] = pm25_component_of(ds,'no3')

In [6]:
#Black carbon organic carbon BC OC
#sum up components
ds['oc_pm25'] = pm25_component_of(ds,'oc')
ds['bc_pm25'] = pm25_component_of(ds,'bc')

In [7]:
#PM2.5 Secondary Organic Aerosols components (SOA)
ds['pm25_bio_soa'] = pm25_component_of(ds,'biog1_o') + pm25_component_of(ds,'biog1_c')  # biogenic from isoprene and pinenes
ds['pm25_bb_soa']  =  pm25_component_of(ds,'smpbb') #biomass burning
ds['pm25_anthro_soa'] = pm25_component_of(ds,'smpa')  #anthropogenic
ds['pm25_gly_soa'] = pm25_component_of(ds,'glysoa_sfc') #glyoxal

ds['pm25_soa'] = (ds['pm25_bio_soa'] + ds['pm25_bb_soa'] + ds['pm25_anthro_soa']+ ds['pm25_gly_soa']) #total PM2.5 SOA ug/m3

In [8]:
#Total PM2.5_dry in Ug/m3 divide by ALT
ds['pm25_dust'] = pm25_component_of(ds,'oin')
ds['pm25_seas'] = pm25_component_of(ds,'na')

ds['pm25_tot_dry'] = (ds['pm25_soa'] + ds['oc_pm25'] + ds['bc_pm25'] + ds['so4_pm25'] +ds['nh4_pm25'] + ds['no3_pm25'] +ds['pm25_dust']+ ds['pm25_seas'])/ds.ALT

In [9]:
ds['P_mb']= (ds['P'] +ds['PB'])*0.01  #total pressure in hPa

In [10]:
def space_subset(dataset, lat_lim, long_lim ):
    """
    Extract spatial subset of data given lat and long limits.
    
    
    """
    
    s_subset= ds.where((long_lim[0] < ds.XLONG) & (ds.XLONG < long_lim[1])
         & (lat_lim[0] < ds.XLAT) & (ds.XLAT < lat_lim[1]), drop=True)
  
    return s_subset

In [11]:
space_subset = space_subset(ds,[20,35], [60,90])  #delhi ncr

In [12]:
#POA
space_subset['oc_pm25'] = space_subset['oc_pm25']/space_subset.ALT   #convert to ug/m 3
space_subset['oc_pm25'].attrs['units']= 'ug/m3'
#BC
space_subset['bc_pm25'] = space_subset['bc_pm25']/space_subset.ALT   #convert to ug/m 3
space_subset['bc_pm25'].attrs['units']= 'ug/m3'

In [13]:
#SOA tot
space_subset['pm25_soa'] = space_subset['pm25_soa']/space_subset.ALT   #convert to ug/m 3
space_subset['pm25_soa'].attrs['units']= 'ug/m3'

In [14]:
#SOA inorg
space_subset['so4_pm25'] = space_subset['so4_pm25']/space_subset.ALT
space_subset['so4_pm25'].attrs['units']= 'ug/m3'
space_subset['nh4_pm25'] = space_subset['nh4_pm25']/space_subset.ALT
space_subset['nh4_pm25'].attrs['units']= 'ug/m3'
space_subset['no3_pm25'] = space_subset['no3_pm25']/space_subset.ALT
space_subset['no3_pm25'].attrs['units']= 'ug/m3'

space_subset['tot_SIA_pm25'] = space_subset['no3_pm25'] + space_subset['nh4_pm25'] +space_subset['so4_pm25']
space_subset['tot_SIA_pm25'].attrs['units']= 'ug/m3'

In [15]:
#PM2.5 Secondary Organic Aerosols components (SOA) ug/m3
space_subset['pm25_bio_soa'] = space_subset['pm25_bio_soa']/space_subset.ALT
space_subset['pm25_anthro_soa'] = space_subset['pm25_anthro_soa']/space_subset.ALT
space_subset['pm25_gly_soa'] = space_subset['pm25_gly_soa']/space_subset.ALT
space_subset['pm25_bb_soa'] = space_subset['pm25_bb_soa']/space_subset.ALT
space_subset['pm25_bio_soa'].attrs['units']= 'ug/m3'
space_subset['pm25_anthro_soa'].attrs['units']= 'ug/m3'
space_subset['pm25_gly_soa'].attrs['units']= 'ug/m3'
space_subset['pm25_bb_soa'].attrs['units']= 'ug/m3'

In [16]:
space_subset['pm25_dust'] = space_subset['pm25_dust']/space_subset.ALT
space_subset['pm25_seas'] = space_subset['pm25_seas']/space_subset.ALT
space_subset['pm25_dust'].attrs['units']= 'ug/m3'
space_subset['pm25_seas'].attrs['units']= 'ug/m3'

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


def var_plot_2D(dataset, var_name, level, title):
    
    var = dataset[var_name]
    
    #draw map
    ax = plt.subplot(projection=ccrs.PlateCarree())
    
    # draw coastlines and borders
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, lw=0.5)
    
    #draw meridians and parallels
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='k', alpha=0.4, linestyle='-')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 10, 'color': 'gray'}
    gl.ylabel_style = {'size': 10, 'color': 'grey'}
    
    #plot SURFACE data
    long = dataset.XLONG.values[0,:,:]
    lat  = dataset.XLAT.values[0,:,:]
    var_values= var[level,:,:] 
    res = 15 #controls the resolution for the map plotting.
    
    cs=plt.contourf( long, lat, var_values, res,
             transform=ccrs.PlateCarree(), cmap=plt.cm.YlGnBu)

    #colorbar
    cbar = plt.colorbar(cs) # pad=0.2 #format='%.2e'
    cbar.set_label(var.units)
    ax.set_title(title)
    
    #plotting parameters
    plt.rcParams['figure.figsize'] = [10, 5]


In [17]:
s_avg = var_space_mean(space_subset)

In [18]:
def timeavg(ds):
     return xr.Dataset(dict(ds.mean(dim='Time', keep_attrs=True).data_vars), coords = dict(ds.coords))

In [19]:
t_avg = timeavg(s_avg)

In [20]:
t_avg.PM2_5_DRY.values

array([103.976776 ,  94.90666  ,  86.31181  ,  75.64267  ,  63.51438  ,
        52.19705  ,  42.53342  ,  34.513466 ,  28.222849 ,  23.211248 ,
        18.907614 ,  14.880034 ,  11.42492  ,   8.604134 ,   6.437525 ,
         5.0186806,   3.995853 ,   3.4233274,   2.9330742,   2.8325615,
         2.4716568,   2.3719058,   2.232597 ,   1.8123615,   1.6708694,
         1.5512154,   1.4200778,   1.4711262,   1.4248192,   1.6657046,
         1.9011167,   3.1005359,   5.7966967,  12.284899 ,  22.401114 ,
        40.15029  , 132.49254  , 306.46692  , 340.6918   ], dtype=float32)

In [26]:
np.savetxt("/exports/csce/datastore/geos/users/s1878599/WRFchem4.0/model/WRF/pm25.csv", t_avg.PM2_5_DRY.values, delimiter=",")

In [27]:
np.savetxt("/exports/csce/datastore/geos/users/s1878599/WRFchem4.0/model/WRF/pressure.csv", t_avg.P_HYD.values, delimiter=",")

In [23]:
def plot_vertical(t_avg,var):
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x= t_avg[var], y= t_avg.P_mb,
                    mode='markers',
                    )).invert_yaxis()
    # Edit the layout
    fig.update_layout(title='Vertical profile',
                   xaxis_title=var +' '+ t_avg[var].units,
                   yaxis_title='pressure hPa')
    fig.show()

In [24]:
plot_vertical(t_avg, 'PM2_5_DRY')

KeyboardInterrupt: 

In [ ]:
#Plot over hydrotatic pressure
    fig = go.Figure()
    fig.add_trace(go.Scatter(x= t_avg['PM2_5_DRY'], y= t_avg.P_HYD,
                    mode='markers',
                    ))
    # Edit the layout
    fig.update_layout(title='Vertical profile',
                   xaxis_title='PM2_5_DRY' +' '+ t_avg['PM2_5_DRY'].units,
                   yaxis_title='hydrostatic pressurePa')
    fig.show()

In [ ]:
t_avg.P_HYD.values

In [ ]:
t_avg.P_mb.values

In [ ]:
#Plot over number of layers
    fig = go.Figure()
    fig.add_trace(go.Scatter(x= t_avg['PM2_5_DRY'], y= t_avg.bottom_top,
                    mode='markers',
                    ))
    # Edit the layout
    fig.update_layout(title='Vertical profile',
                   xaxis_title='PM2_5_DRY' +' '+ t_avg['PM2_5_DRY'].units,
                   yaxis_title='levels')
    fig.show()